# Building a Factorization Machine in TensorFlow

Creating a sparse matrix which is needed for feeding in

In [1]:
import numpy as np
import pandas as pd

In [2]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr

def vectorize_dic(dic, ix=None, p=None):
    """ 
    Creates a scipy csr matrix from a list of lists (each inner list is a set of values corresponding to a feature) 
    
    parameters:
    -----------
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if (ix == None):
        ix = defaultdict(count(0).__next__)
        
    n = len(dic.values()[0]) # num samples
    g = len(dic.keys()) # num groups
    nz = n * g # number of non-zeros

    col_ix = np.empty(nz, dtype=int)
    
    i = 0
    for k, lis in dic.iteritems():
        # append index el with k in order to prevet mapping different columns with same id to same index
        col_ix[i::g] = [ix[str(el) + str(k)] for el in lis]
        i += 1
        
    row_ix = np.repeat(np.arange(0, n), g)
    data = np.ones(nz)
    
    if (p == None):
        p = len(ix)
        
    ixx = np.where(col_ix < p)

    return csr.csr_matrix((data[ixx],(row_ix[ixx], col_ix[ixx])), shape=(n, p)), ix

Reacing in data

Limit row for first run

Test Train Split

Import libraries

In [53]:
X_train = np.loadtxt('fm_Xtrain.txt')
X_test = np.loadtxt('fm_Xtest.txt')
y_train = np.loadtxt('fm_ytrain.txt')
y_test = np.loadtxt('fm_ytest.txt')

In [52]:
train.shape
# X_train = train
# print X_test.shape

In [54]:
import tensorflow as tf

n, p = X_train.shape

# number of latent factors
k = 100

# design matrix
X = tf.placeholder('float', shape=[None, p])
# target vector
y = tf.placeholder('float', shape=[None, 1])

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimate of y, initialized to 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

In [55]:

# Calculate output with FM equation
linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keep_dims=True))
pair_interactions = (tf.multiply(0.5,
                    tf.reduce_sum(
                        tf.subtract(
                            tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                            tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                        1, keep_dims=True)))
y_hat = tf.add(linear_terms, pair_interactions)

In [56]:

# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

In [57]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)


In [58]:
def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)

In [61]:
from tqdm import tqdm_notebook as tqdm
import time

epochs = 10
batch_size = 1000

# Launch the graph
init = tf.global_variables_initializer()
sess = tf.Session()

start_time_total = time.time()
sess.run(init)

for epoch in tqdm(range(epochs), unit='epoch'):
    perm = np.random.permutation(X_train.shape[0])
    # iterate over batches
    for bX, bY in batcher(X_train[perm], y_train[perm], batch_size):
        sess.run(optimizer, feed_dict={X: bX.reshape(-1, p), y: bY.reshape(-1, 1)})

print("\nTotal Run Time")
print(time.time() - start_time_total, "seconds")

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))



Total Run Time
66.91984987258911 seconds


In [62]:
errors = []
for bX, bY in batcher(X_test, y_test):
    errors.append(sess.run(error, feed_dict={X: bX.reshape(-1, p), y: bY.reshape(-1, 1)}))

RMSE = np.sqrt(np.array(errors).mean())
print(RMSE)

1.1028684
